# MSc Data Science Dissertation

## Sample Dataset Analysis

#### Name: Chandana Karunaratne
#### Student ID: 1621633
#### Date: 24 August 2017

In [129]:
# The code in this file is used to analyze the travel patterns of passengers using the following steps:

# Step 1: Read in the sample data on TfL passenger journeys
# Step 2: Clean and prepare the data so that the time and the date on which each journey takes place is recoded to facilitate analysis
# Step 3: Create a dataframe of unique passengers and their respective most frequently used station of entry
# Step 4: Assign each passenger his most likely socio-demographic characteristics based on his 
# approximated place of residence (proxied by his most frequently used station of entry)
# Step 5: Conduct cluster analysis on this dataset of unique passengers and their corresponding socio-demographic characteristics
# Step 6: Obtain information on the travel patterns of each cluster, including the most frequently used stations,
# the distribution of travel throughout a day, and the distribution of travel throughout a week.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import patsy
import statsmodels.formula.api as sm


## Step 1: Read in the sample data on TfL passenger journeys

In [3]:
# Read in the sample data (they are in parts because creating csv files for the entire sample at once took too much memory)
# These files were obtained by getting a random sample of records for each day in the Spark database

sample1_df = pd.read_csv("Full_Sample_Part1.csv")
sample2_df = pd.read_csv("Full_Sample_Part2.csv")
sample3_df = pd.read_csv("Full_Sample_Part3.csv")
sample4_df = pd.read_csv("Full_Sample_Part4.csv")
sample5_df = pd.read_csv("Full_Sample_Part5.csv")
sample6_df = pd.read_csv("Full_Sample_Part6.csv")
sample7_df = pd.read_csv("Full_Sample_Part7.csv")
sample8_df = pd.read_csv("Full_Sample_Part8.csv")


In [4]:
# Concatenate all datafromes into one large dataframe which contains all randomly sampled records

frames = [sample1_df, sample2_df, sample3_df, sample4_df, sample5_df, sample6_df, sample7_df, sample8_df]

final_df = pd.concat(frames)


In [5]:
# Show the head of the final_df dataframe

final_df.head()


,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME
0,11170717,-1,12393,15,247,-1,896
1,31776450,5,12393,15,213,111,1483
2,6666492,-1,12393,15,76,-1,1126
3,58927301,-1,12393,15,158,-1,1333
4,23724328,-1,12393,15,193,-1,1021


## Step 2: Clean and prepare the data so that the time and the date on which each journey takes place is recoded to facilitate analysis

In [6]:
# Create a new column called 'Timeblock' and assign null values for the timebeing.

final_df['Timeblock'] = np.nan


In [7]:
# Assign a timeblock to each value of TRANSACTIONTIME so that each falls into a 2-hour time interval throughout a 24-hour day

# Source: https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns?noredirect=1&lq=1

def timeblock(row):
    if (row['TRANSACTIONTIME'] >= 0 and row['TRANSACTIONTIME'] < 120):
        return 1
    elif (row['TRANSACTIONTIME'] >= 120 and row['TRANSACTIONTIME'] < 240):
        return 2
    elif (row['TRANSACTIONTIME'] >= 240 and row['TRANSACTIONTIME'] < 360):
        return 3
    elif (row['TRANSACTIONTIME'] >= 360 and row['TRANSACTIONTIME'] < 480):
        return 4
    elif (row['TRANSACTIONTIME'] >= 480 and row['TRANSACTIONTIME'] < 600):
        return 5
    elif (row['TRANSACTIONTIME'] >= 600 and row['TRANSACTIONTIME'] < 720):
        return 6
    elif (row['TRANSACTIONTIME'] >= 720 and row['TRANSACTIONTIME'] < 840):
        return 7
    elif (row['TRANSACTIONTIME'] >= 840 and row['TRANSACTIONTIME'] < 960):
        return 8
    elif (row['TRANSACTIONTIME'] >= 960 and row['TRANSACTIONTIME'] < 1080):
        return 9
    elif (row['TRANSACTIONTIME'] >= 1080 and row['TRANSACTIONTIME'] < 1200):
        return 10
    elif (row['TRANSACTIONTIME'] >= 1200 and row['TRANSACTIONTIME'] < 1320):
        return 11
    elif (row['TRANSACTIONTIME'] >= 1320 and row['TRANSACTIONTIME'] < 1440):
        return 12
    else:
        return 99      # assign the value 99 for transaction times that fall outside the 24-hour window


final_df['Timeblock'] = final_df.apply(lambda row: timeblock (row), axis=1)
    
    

In [8]:
# Show the head of the final_df dataframe after assigning a timeblock to each transaction time

final_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock
0,11170717,-1,12393,15,247,-1,896,8
1,31776450,5,12393,15,213,111,1483,99
2,6666492,-1,12393,15,76,-1,1126,10
3,58927301,-1,12393,15,158,-1,1333,12
4,23724328,-1,12393,15,193,-1,1021,9


In [9]:
# Create an array of the unique days in the dataset

day_array = final_df['DAYKEY'].unique()

day_array

array([12393, 12386, 12400, 12399, 12391, 12398, 12392, 12385, 12379,
       12372, 12383, 12378, 12397, 12390, 12370, 12365, 12356, 12377,
       12355, 12351, 12357, 12363, 12369, 12358, 12350, 12362, 12348,
       12349, 12396, 12389, 12382, 12368, 12375, 12347, 12361, 12354,
       12387, 12401, 12394, 12373, 12366, 12359, 12352, 12380, 12395,
       12388, 12353, 12367, 12381, 12374, 12360, 12346, 12376, 12384,
       12371, 12364], dtype=int64)

In [10]:
# Check if there are any null values in the 'DAYKEY' column

final_df['DAYKEY'].isnull().sum()


0

In [11]:
# Match each daykey with its corresponding day of the week

columns = ['DAYKEY', 'Day of the Week']

index = np.arange(56)

day_df = pd.DataFrame(columns = columns, index = index)

count = 0

while count < len(day_array):
    if (day_array[count] == 12347 or day_array[count] == 12354 or day_array[count] ==  12361 or day_array[count] ==  12368 or day_array[count] == 12375 or day_array[count] == 12382 or day_array[count] == 12389 or day_array[count] == 12396):
        day_df['DAYKEY'][count] = day_array[count]
        day_df['Day of the Week'][count] = 'Monday'
    elif (day_array[count] == 12348 or day_array[count] == 12355 or day_array[count] == 12362 or day_array[count] == 12369 or day_array[count] == 12376 or day_array[count] == 12383 or day_array[count] == 12390 or day_array[count] == 12397):
        day_df['DAYKEY'][count] = day_array[count]
        day_df['Day of the Week'][count] = 'Tuesday'
    elif (day_array[count] == 12349 or day_array[count] == 12356 or day_array[count] == 12363 or day_array[count] == 12370 or day_array[count] == 12377 or day_array[count] == 12384 or day_array[count] == 12391 or day_array[count] == 12398):
        day_df['DAYKEY'][count] = day_array[count]
        day_df['Day of the Week'][count] = 'Wednesday'
    elif (day_array[count] == 12350 or day_array[count] == 12357 or day_array[count] == 12364 or day_array[count] == 12371 or day_array[count] == 12378 or day_array[count] == 12385 or day_array[count] == 12392 or day_array[count] == 12399):
        day_df['DAYKEY'][count] = day_array[count]
        day_df['Day of the Week'][count] = 'Thursday'
    elif (day_array[count] == 12351 or day_array[count] == 12358 or day_array[count] == 12365 or day_array[count] == 12372 or day_array[count] == 12379 or day_array[count] == 12386 or day_array[count] == 12393 or day_array[count] == 12400):
        day_df['DAYKEY'][count] = day_array[count]
        day_df['Day of the Week'][count] = 'Friday'
    elif (day_array[count] == 12345 or day_array[count] == 12352 or day_array[count] == 12359 or day_array[count] == 12366 or day_array[count] == 12373 or day_array[count] == 12380 or day_array[count] == 12387 or day_array[count] == 12394 or day_array[count] == 12401):
        day_df['DAYKEY'][count] = day_array[count]
        day_df['Day of the Week'][count] = 'Saturday'
    else:
        day_df['DAYKEY'][count] = day_array[count]
        day_df['Day of the Week'][count] = 'Sunday'
    count = count + 1



In [12]:
# Show the day_df dataframe

day_df

,DAYKEY,Day of the Week
0,12393,Friday
1,12386,Friday
2,12400,Friday
3,12399,Thursday
4,12391,Wednesday
5,12398,Wednesday
6,12392,Thursday
7,12385,Thursday
8,12379,Friday
9,12372,Friday


In [13]:
# Merge the final_df dataframe with the day_df dataframe so that you assign days of the week (e.g. Monday, Tuesday, etc.)
# to each value of DAYKEY in the final_df dataframe

finalmerged_df = final_df.merge(day_df, left_on='DAYKEY', right_on='DAYKEY', how='left')


In [14]:
# Show the head of the finalmerged_df dataframe

finalmerged_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock,Day of the Week
0,11170717,-1,12393,15,247,-1,896,8,Friday
1,31776450,5,12393,15,213,111,1483,99,Friday
2,6666492,-1,12393,15,76,-1,1126,10,Friday
3,58927301,-1,12393,15,158,-1,1333,12,Friday
4,23724328,-1,12393,15,193,-1,1021,9,Friday


## Step 3: Create a dataframe of unique passengers and their respective most frequently used station of entry

In [15]:
# Find the most frequently used station of entry for each prestigeid:
# The code below does a 'groupby' on PRESTIGEID for the STATIONOFFIRSTENTRYKEY column, gets a count for the values in 
# each group in descending order, and then takes the first row for each group.

# SOURCE: https://stackoverflow.com/questions/23692419/python-select-most-frequent-using-group-by

station_mode = finalmerged_df.groupby('PRESTIGEID')['STATIONOFFIRSTENTRYKEY'].agg(lambda x: x.value_counts().index[0])


In [16]:
# Convert the station_mode series to a dataframe.

passenger_df = pd.DataFrame({'PRESTIGEID': station_mode.index, 'STATIONOFFIRSTENTRYKEY': station_mode.values})


In [17]:
# Show the head of the passenger_df dataframe

passenger_df.head()


,PRESTIGEID,STATIONOFFIRSTENTRYKEY
0,125,161
1,190,70
2,247,1980
3,355,1955
4,376,1968


## Step 4: Assign each passenger his most likely socio-demographic characteristics based on his approximated place of residence (proxied by his most frequently used station of entry)

### Preparing LOAC dataframe:

In [18]:
# Read in a file which contains the Supergroup code for each TfL tube and rail station (prepared in a separate Jupyter Notebook file)

loac_input_df = pd.read_csv("Stations with LOAC Code_FINAL.csv")

In [19]:
# Show the head of the loac_input_df dataframe

loac_input_df.head()

,id,STATIONKEY,STATIONNAME,Keep,Supergroup
0,1,1,"Acton Town Station, England",True,B
1,2,2,"Barbican Station, England",True,F
2,3,3,"Aldgate Station, England",True,D
3,4,4,"Aldgate East Station, England",True,D
4,5,5,"Alperton Station, England",True,C


In [20]:
# Delete the string 'Station, London' from the end of each Station Name because it is no longer needed.

import re

count = 0

while count < len(loac_input_df):
    loac_input_df['STATIONNAME'][count] = re.sub('\ Station, England$', '', loac_input_df['STATIONNAME'][count])
    count = count + 1



C:\Users\HP-PC\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
# Merge the passengerfinal_df dataframe with the loac_input_df dataframe so that you get a dataframe with unique passengers and
# their corresponding Supergroup code based on their most frequently used station of entry

passengerfinal_df = passenger_df.merge(loac_input_df, left_on='STATIONOFFIRSTENTRYKEY', right_on='STATIONKEY', how='left')


In [22]:
# Show the head of the passengerfinal_df dataframe, which contains all unique passenger id's (PRESTIGEID's) and their 
# respective most frequently used station of entry and corresponding Supergroup code

passengerfinal_df.head()


,PRESTIGEID,STATIONOFFIRSTENTRYKEY,id,STATIONKEY,STATIONNAME,Keep,Supergroup
0,125,161,154,161,Plaistow,True,G
1,190,70,68,70,Euston Square,True,D
2,247,1980,366,1980,Richmond,True,D
3,355,1955,349,1955,Gunnersbury,True,C
4,376,1968,360,1968,Barking,True,B


In [23]:
# Check if there are any null values

passengerfinal_df.isnull().values.any()


False

### Merge the new Passenger dataframe with the LOAC traits dataframe:

In [24]:
# Read in the CSV file with the socio-economic traits for each Supergroup

traits_df = pd.read_csv("LOAC_SocEc_Traits_CK2.csv")


In [25]:
# Show the head of the traits_df dataframe

traits_df.head()

,Supergroup,% households where no one speaks English as a main language,% persons aged 0 to 4,% persons aged 5 to 14,% persons aged 25 to 44,% persons aged 45 to 64,% persons aged 65 to 89,% persons aged 90 and over,% persons born in UK or Ireland,% persons who are divorced or separated,...,% persons who live in a terrace or end-terrace house,% households with two or more motor vehicles,% persons whose highest qualification is Level 3,"% persons whose highest qualification is Level 1, Level 2, or Apprenticeship",% persons whose highest qualification is Level 4 or above,% schoolchildren and full-time students aged 16 and over,% working age adults who use private transport to get to work,% working age adults who use public transport to get to work,"% working age adults who walk, cycle, or use an alternative method to get to work",Standardised Illness Ratio
0,A,-1.8,0.6,2.4,-6.9,1.9,1.6,0.2,13.9,2.6,...,17.5,4.1,0.2,9.3,-15.6,-1.6,9.6,-8.1,-2.9,19.1
1,B,2.6,1.2,2.4,-0.9,-2.6,-2.0,-0.1,-7.4,4.0,...,-19.6,-14.3,0.0,1.0,-9.4,3.1,-10.6,-0.4,1.7,25.7
2,C,3.1,-0.1,1.4,-4.1,1.1,0.4,-0.1,-12.3,-2.7,...,11.3,10.8,0.0,1.6,-6.5,2.0,6.8,-4.8,-3.3,-1.0
3,D,-1.6,-2.0,-6.7,18.4,-5.8,-4.0,-0.2,-13.6,-2.4,...,-18.0,-11.7,-0.8,-12.9,24.6,1.2,-11.3,11.5,7.4,-36.4
4,E,-0.9,-0.5,-2.6,7.9,-2.0,-2.2,-0.1,1.2,0.0,...,-1.8,-10.1,-0.3,-5.6,10.3,-0.8,-8.2,8.3,3.8,1.4


In [26]:
# Merge the passengerfinal_df with the traits_df so that we assign socio-economic characteristics corresponding to each
# Supergroup code for each passenger

passengerfinal_df2 = passengerfinal_df.merge(traits_df, left_on='Supergroup', right_on='Supergroup', how='left')


In [27]:
# Show the head of the passengerfinal_df2 dataframe

passengerfinal_df2.head()

,PRESTIGEID,STATIONOFFIRSTENTRYKEY,id,STATIONKEY,STATIONNAME,Keep,Supergroup,% households where no one speaks English as a main language,% persons aged 0 to 4,% persons aged 5 to 14,...,% persons who live in a terrace or end-terrace house,% households with two or more motor vehicles,% persons whose highest qualification is Level 3,"% persons whose highest qualification is Level 1, Level 2, or Apprenticeship",% persons whose highest qualification is Level 4 or above,% schoolchildren and full-time students aged 16 and over,% working age adults who use private transport to get to work,% working age adults who use public transport to get to work,"% working age adults who walk, cycle, or use an alternative method to get to work",Standardised Illness Ratio
0,125,161,154,161,Plaistow,True,G,3.3,1.5,1.9,...,6.5,-6.8,-0.2,1.6,-9.3,2.8,-2.5,0.7,-2.4,5.3
1,190,70,68,70,Euston Square,True,D,-1.6,-2.0,-6.7,...,-18.0,-11.7,-0.8,-12.9,24.6,1.2,-11.3,11.5,7.4,-36.4
2,247,1980,366,1980,Richmond,True,D,-1.6,-2.0,-6.7,...,-18.0,-11.7,-0.8,-12.9,24.6,1.2,-11.3,11.5,7.4,-36.4
3,355,1955,349,1955,Gunnersbury,True,C,3.1,-0.1,1.4,...,11.3,10.8,0.0,1.6,-6.5,2.0,6.8,-4.8,-3.3,-1.0
4,376,1968,360,1968,Barking,True,B,2.6,1.2,2.4,...,-19.6,-14.3,0.0,1.0,-9.4,3.1,-10.6,-0.4,1.7,25.7


In [28]:
# Check the frequency distribution of the 'Supergroup' column (in terms of proportion)

passengerfinal_df2.Supergroup.value_counts(normalize=True)


D    0.476116
B    0.124819
G    0.119033
E    0.116379
F    0.078155
C    0.056622
X    0.015643
A    0.008526
H    0.004707
Name: Supergroup, dtype: float64

## Step 5: Conduct cluster analysis on the dataset of unique passengers and their corresponding socio-demographic characteristics

In [30]:

import sklearn
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn import metrics


In [31]:
# This deletes non-essential columns from the passenger_df dataframe and prepares it for k-means clustering

passengerfinal_df_clustering = passengerfinal_df2.drop('STATIONNAME', axis=1)
passengerfinal_df_clustering = passengerfinal_df_clustering.drop('Keep', axis=1)


In [32]:
# Show the head of the passengerfinal_df_clustering dataframe

passengerfinal_df_clustering.head()


,PRESTIGEID,STATIONOFFIRSTENTRYKEY,id,STATIONKEY,Supergroup,% households where no one speaks English as a main language,% persons aged 0 to 4,% persons aged 5 to 14,% persons aged 25 to 44,% persons aged 45 to 64,...,% persons who live in a terrace or end-terrace house,% households with two or more motor vehicles,% persons whose highest qualification is Level 3,"% persons whose highest qualification is Level 1, Level 2, or Apprenticeship",% persons whose highest qualification is Level 4 or above,% schoolchildren and full-time students aged 16 and over,% working age adults who use private transport to get to work,% working age adults who use public transport to get to work,"% working age adults who walk, cycle, or use an alternative method to get to work",Standardised Illness Ratio
0,125,161,154,161,G,3.3,1.5,1.9,0.3,-2.5,...,6.5,-6.8,-0.2,1.6,-9.3,2.8,-2.5,0.7,-2.4,5.3
1,190,70,68,70,D,-1.6,-2.0,-6.7,18.4,-5.8,...,-18.0,-11.7,-0.8,-12.9,24.6,1.2,-11.3,11.5,7.4,-36.4
2,247,1980,366,1980,D,-1.6,-2.0,-6.7,18.4,-5.8,...,-18.0,-11.7,-0.8,-12.9,24.6,1.2,-11.3,11.5,7.4,-36.4
3,355,1955,349,1955,C,3.1,-0.1,1.4,-4.1,1.1,...,11.3,10.8,0.0,1.6,-6.5,2.0,6.8,-4.8,-3.3,-1.0
4,376,1968,360,1968,B,2.6,1.2,2.4,-0.9,-2.6,...,-19.6,-14.3,0.0,1.0,-9.4,3.1,-10.6,-0.4,1.7,25.7


In [33]:
# Check if the new dataframe has any null values

passengerfinal_df_clustering.isnull().values.any()


True

In [34]:
# Check which columns have null values and how many each column has

passengerfinal_df_clustering.isnull().sum()


PRESTIGEID                                                                                0
STATIONOFFIRSTENTRYKEY                                                                    0
id                                                                                        0
STATIONKEY                                                                                0
Supergroup                                                                                0
% households where no one speaks English as a main  language                          34326
% persons aged 0 to 4                                                                 34326
% persons aged 5 to 14                                                                34326
% persons aged 25 to 44                                                               34326
% persons aged 45 to 64                                                               34326
% persons aged 65 to 89                                                         

In [35]:
# Drop rows with null values, since the null values relate to those records in which a Supergroup code was not assigned
# (these rows are designated by the Supergroup code "X"), and as such, these records in which a Supergroup code was not 
# assigned are no longer relevant for the next stage of our analysis

passengerfinal_df_clustering = passengerfinal_df_clustering.dropna()


In [36]:
# Based on sd-clean.py by Elizabeth Sklar (28-mar-2017)

# Input the passengerfinal_df_clustering dataframe into a list in preparation for the k-means clustering


import sys
import csv

passengerfinal_df_clustering.to_csv("passengerfinal_df_clustering.csv", sep = ',')

try:
# open data file in csv format
    f = open( 'passengerfinal_df_clustering.csv', 'rU' )
# read contents of data file into "rawdata" list
    indata = csv.reader( f )
# parse data in csv format
    rawdata = [rec for rec in indata]
# handle exceptions:
except IOError as iox:
    print '** I/O error trying to open the data file> ' + str( iox )
    sys.exit()
except Exception as x:
    print '** error> ' + str( x )
    sys.exit()


In [37]:
# Based on sd-clean.py by Elizabeth Sklar (28-mar-2017)

# The code below handles the header in the input file by printing it and then deleting it from the data file

header = rawdata[0]
del rawdata[0]
print 'header='
print header
print( 'number of features = %d' % len( header ) )
print 'features:'
for i, f in zip( range(len(header)), header ):
    print( 'index=%d  feature=[%s]' % ( i, f ))


header=
['', 'PRESTIGEID', 'STATIONOFFIRSTENTRYKEY', 'id', 'STATIONKEY', 'Supergroup', '% households where no one speaks English as a main  language', '% persons aged 0 to 4', '% persons aged 5 to 14', '% persons aged 25 to 44', '% persons aged 45 to 64', '% persons aged 65 to 89', '% persons aged 90 and over', '% persons born in UK or Ireland', '% persons who are divorced or separated', '% persons who are married or in a civil partnership', '% persons who are single', '% persons who have Arab or any other ethnicity', '% persons who have Bangladeshi ethnicity', '% persons who have Black ethnicity', '% persons who have Chinese or Other Asian ethnicity', '% persons who have Indian ethnicity', '% persons who have mixed ethnicity', '% persons who have Pakistani ethnicity', '% persons who have White ethnicity', 'Number of persons per hectare', '% working age adults who are unemployed', '% working age adults who work full-time', '% working age adults who work in accommodation or food service

In [ ]:
# The following cells conduct cluster analysis on each set of related features

### Features related to Age

In [97]:
features = [7, 8, 9, 10, 11, 12]

num_features = len(features)

label = 5

#-gather fields of interest from data set into X and y
X = []
y = []

for rec in rawdata:
    instance = []
    for f in features:
        instance.append( float( rec[f] ))
    X.append( instance )        # Get each record
    y.append( rec[label] )      # Get the corresponding Supergroup label of each record
        
X = np.array( X )
y = np.array( y )

num_instances = X.shape[0]

print 'number of instances = %d' % num_instances
print 'shape of input data = %d x %d' % ( X.shape[0], X.shape[1] )
print 'shape of target data = %d' % ( y.shape[0] )



number of instances = 2160016
shape of input data = 2160016 x 6
shape of target data = 2160016


In [92]:
# Create a dataframe with the results of the Inertia and Calinski-Harabasz scores


columns = ['No. of clusters', 'Inertia', 'Calinski-Harabaz Score']

index = np.arange(14)

df1 = pd.DataFrame(columns = columns, index = index)

line_count = 0

n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 100]

cluster_count = 0

while cluster_count < 14:
    km = cluster.KMeans( n_clusters[cluster_count] ).fit( X )
    labels = km.labels_
    ch_score = metrics.calinski_harabaz_score(X, labels)
    df1['No. of clusters'][line_count] = n_clusters[cluster_count]
    df1['Inertia'][line_count] = km.inertia_
    df1['Calinski-Harabaz Score'][line_count] = ch_score
    cluster_count = cluster_count + 1
    line_count = line_count + 1



In [106]:
# Show the results of the Inertia and Calinski-Harabasz scores

df1

,No. of clusters,Inertia,Calinski-Harabaz Score
0,2,3.35668e+07,1.33623e+07
1,3,1.44119e+07,1.69966e+07
2,4,5.95575e+06,2.84415e+07
3,5,2.17439e+06,5.93659e+07
4,6,513212,2.02617e+08
5,7,186988,4.64052e+08
6,8,1.07352e-14,2.46875e+27
7,9,1.07352e-14,2.46875e+27
8,10,1.07352e-14,2.46875e+27
9,20,7.68617e-16,2.46875e+27


In [98]:
# Since the above results show that the ideal clustering is 8 clusters, fit the model such that it partitions the data into
# 8 clusters.

km = cluster.KMeans(n_clusters=8).fit(X)

In [99]:
# Show cluster centers (centroids) indicating mean values of each feature for each cluster

km.cluster_centers_

array([[ -2. ,  -6.7,  18.4,  -5.8,  -4. ,  -0.2],
       [  1.2,   2.4,  -0.9,  -2.6,  -2. ,  -0.1],
       [ -0.5,  -2.6,   7.9,  -2. ,  -2.2,  -0.1],
       [  0.3,  -1.3,   1.1,   1.9,   1.8,   0.3],
       [ -0.1,   1.4,  -4.1,   1.1,   0.4,  -0.1],
       [ -1.9,   0.9, -13.3,   7.8,   7.4,   0.3],
       [  1.5,   1.9,   0.3,  -2.5,  -2.8,  -0.2],
       [  0.6,   2.4,  -6.9,   1.9,   1.6,   0.2]])

In [100]:
# Show cluster labels

my_label = km.labels_

my_label


array([6, 0, 0, ..., 0, 1, 0])

In [101]:
# Show distribution of cluster labels

np.unique(my_label, return_counts=True)


(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([1044761,  273896,  255376,  171498,  124249,   10328,  261200,
          18708], dtype=int64))

In [102]:
# Show distribution of Supergroup labels

np.unique(y, return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 
       dtype='|S1'),
 array([  18708,  273896,  124249, 1044761,  255376,  171498,  261200,
          10328], dtype=int64))

### Features related to Marital Status

In [103]:
features = [14, 15, 16]

num_features = len(features)

#-gather fields of interest from full data set into X and y
X = []

for rec in rawdata:
    instance = []
    for f in features:
        instance.append( float( rec[f] ))
    X.append( instance )
        
X = np.array( X )
num_instances = X.shape[0]

print 'number of instances = %d' % num_instances
print 'shape of input data = %d x %d' % ( X.shape[0], X.shape[1] )



number of instances = 2160016
shape of input data = 2160016 x 3


In [39]:
# Create a dataframe with the results of the Inertia and Calinski-Harabasz scores


columns = ['No. of clusters', 'Inertia', 'Calinski-Harabaz Score']

index = np.arange(14)

df2 = pd.DataFrame(columns = columns, index = index)

line_count = 0

n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 100]

cluster_count = 0

while cluster_count < 14:
    km = cluster.KMeans( n_clusters[cluster_count] ).fit( X )
    labels = km.labels_
    ch_score = metrics.calinski_harabaz_score(X, labels)
    df2['No. of clusters'][line_count] = n_clusters[cluster_count]
    df2['Inertia'][line_count] = km.inertia_
    df2['Calinski-Harabaz Score'][line_count] = ch_score
    cluster_count = cluster_count + 1
    line_count = line_count + 1



In [40]:
# Show the results of the Inertia and Calinski-Harabasz scores

df2

,No. of clusters,Inertia,Calinski-Harabaz Score
0,2,6.02893e+07,7.27175e+06
1,3,2.58122e+07,9.93482e+06
2,4,1.04958e+07,1.7339e+07
3,5,5.50453e+06,2.52857e+07
4,6,1.64445e+06,6.8726e+07
5,7,712839,1.3259e+08
6,8,1.62552e-14,1.10681e+27
7,9,1.054e-15,1.10681e+27
8,10,1.05338e-15,1.10681e+27
9,20,6.93136e-16,1.10681e+27


In [104]:
# Since the above results show that the ideal clustering is 8 clusters, fit the model such that it partitions the data into
# 8 clusters.

km = cluster.KMeans(n_clusters=8).fit(X)

In [105]:
# Show cluster centers (centroids) indicating mean values of each feature for each cluster

km.cluster_centers_

array([[ -2.40000000e+00,  -1.05000000e+01,   1.56000000e+01],
       [ -2.70000000e+00,   1.15000000e+01,  -9.00000000e+00],
       [  1.80000000e+00,  -2.60000000e+00,   1.20000000e+00],
       [  4.00000000e+00,  -1.10000000e+01,   7.10000000e+00],
       [ -1.40000000e+00,   5.90000000e+00,  -4.60000000e+00],
       [ -2.27107222e-12,  -1.08000000e+01,   1.20000000e+01],
       [ -3.20000000e+00,   1.78000000e+01,  -1.66000000e+01],
       [  2.60000000e+00,   8.00000000e-01,  -5.10000000e+00]])

In [106]:
# Show cluster labels

my_label = km.labels_

my_label


array([2, 0, 0, ..., 0, 3, 0])

In [107]:
# Show distribution of cluster labels

np.unique(my_label, return_counts=True)


(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([1044761,  124249,  261200,  273896,  171498,  255376,   10328,
          18708], dtype=int64))

In [108]:
# Show distribution of Supergroup labels

np.unique(y, return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 
       dtype='|S1'),
 array([  18708,  273896,  124249, 1044761,  255376,  171498,  261200,
          10328], dtype=int64))

### Features related to Ethnicity

In [109]:
features = [17, 18, 19, 20, 21, 22, 23, 24]

num_features = len(features)

#-gather fields of interest from full data set into X and y
X = []

for rec in rawdata:
    instance = []
    for f in features:
        instance.append( float( rec[f] ))
    X.append( instance )
        
X = np.array( X )
num_instances = X.shape[0]

print 'number of instances = %d' % num_instances
print 'shape of input data = %d x %d' % ( X.shape[0], X.shape[1] )



number of instances = 2160016
shape of input data = 2160016 x 8


In [46]:
# Create a dataframe with the results of the Inertia and Calinski-Harabasz scores


columns = ['No. of clusters', 'Inertia', 'Calinski-Harabaz Score']

index = np.arange(14)

df3 = pd.DataFrame(columns = columns, index = index)

line_count = 0

n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 100]

cluster_count = 0

while cluster_count < 14:
    km = cluster.KMeans( n_clusters[cluster_count] ).fit( X )
    labels = km.labels_
    ch_score = metrics.calinski_harabaz_score(X, labels)
    df3['No. of clusters'][line_count] = n_clusters[cluster_count]
    df3['Inertia'][line_count] = km.inertia_
    df3['Calinski-Harabaz Score'][line_count] = ch_score
    cluster_count = cluster_count + 1
    line_count = line_count + 1



In [47]:
# Show the results of the Inertia and Calinski-Harabasz scores

df3

,No. of clusters,Inertia,Calinski-Harabaz Score
0,2,1.33151e+08,7.89934e+06
1,3,6.71268e+07,8.89669e+06
2,4,3.32359e+07,1.27133e+07
3,5,1.11714e+07,2.94341e+07
4,6,579747,4.61634e+08
5,7,103356,2.1595e+09
6,8,2.96572e-14,3.02224e+27
7,9,2.96572e-14,3.02224e+27
8,10,3.8285e-15,3.02224e+27
9,20,2.45657e-15,3.02224e+27


In [110]:
# Since the above results show that the ideal clustering is 8 clusters, fit the model such that it partitions the data into
# 8 clusters.

km = cluster.KMeans(n_clusters=8).fit(X)

In [111]:
# Show cluster centers (centroids) indicating mean values of each feature for each cluster

km.cluster_centers_

array([[  2.1,   6. ,  13.4,  -0.6,  -4. ,   1.6,  -1.4, -17.2],
       [  1.6,  -1.3,  -8.1,   2.1,  -1.4,  -0.1,  -1.4,   8.6],
       [  0.8,   0.4,  -2.7,   5.6,  16.2,  -1.2,   5.8, -24.8],
       [ -0.3,  -1. ,   2. ,  -1.7,  -3.7,   1.4,  -1.6,   4.8],
       [ -1.1,  -2.1,  -9.7,  -1.8,  -2.3,  -0.8,  -1.5,  19.2],
       [  1.1,   1.2,   9.6,   1.8,   0.4,   1.1,   2.5, -17.6],
       [ -1.7,  -1.6,   0.9,  -2.2,  -3.2,  -0.3,  -1.2,   9.3],
       [ -2.2,  -2.1, -10. ,  -2.5,  -0.7,  -2.3,  -1.4,  21.2]])

In [112]:
# Show cluster labels

my_label = km.labels_

my_label


array([5, 1, 1, ..., 1, 0, 1])

In [113]:
# Show distribution of cluster labels

np.unique(my_label, return_counts=True)


(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 273896, 1044761,  124249,  255376,  171498,  261200,   18708,
          10328], dtype=int64))

In [114]:
# Show distribution of Supergroup labels

np.unique(y, return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 
       dtype='|S1'),
 array([  18708,  273896,  124249, 1044761,  255376,  171498,  261200,
          10328], dtype=int64))

### Features related to Employment Sector

In [115]:
features = [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]

num_features = len(features)

#-gather fields of interest from full data set into X and y
X = []

for rec in rawdata:
    instance = []
    for f in features:
        instance.append( float( rec[f] ))
    X.append( instance )
        
X = np.array( X )
num_instances = X.shape[0]

print 'number of instances = %d' % num_instances
print 'shape of input data = %d x %d' % ( X.shape[0], X.shape[1] )



number of instances = 2160016
shape of input data = 2160016 x 13


In [52]:
# Create a dataframe with the results of the Inertia and Calinski-Harabasz scores


columns = ['No. of clusters', 'Inertia', 'Calinski-Harabaz Score']

index = np.arange(14)

df5 = pd.DataFrame(columns = columns, index = index)

line_count = 0

n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 100]

cluster_count = 0

while cluster_count < 14:
    km = cluster.KMeans( n_clusters[cluster_count] ).fit( X )
    labels = km.labels_
    ch_score = metrics.calinski_harabaz_score(X, labels)
    df5['No. of clusters'][line_count] = n_clusters[cluster_count]
    df5['Inertia'][line_count] = km.inertia_
    df5['Calinski-Harabaz Score'][line_count] = ch_score
    cluster_count = cluster_count + 1
    line_count = line_count + 1



In [53]:
# Show the results of the Inertia and Calinski-Harabasz scores

df5

,No. of clusters,Inertia,Calinski-Harabaz Score
0,2,4.89482e+07,9.49267e+06
1,3,1.16806e+07,2.33356e+07
2,4,6.45207e+06,2.87474e+07
3,5,3.49929e+06,4.02095e+07
4,6,1.23135e+06,9.22106e+07
5,7,442266,2.14585e+08
6,8,3.60241e-14,7.68282e+26
7,9,3.60241e-14,7.68282e+26
8,10,3.60238e-14,7.68282e+26
9,20,1.45156e-15,7.68282e+26


In [116]:
# Since the above results show that the ideal clustering is 8 clusters, fit the model such that it partitions the data into
# 8 clusters.

km = cluster.KMeans(n_clusters=8).fit(X)

In [117]:
# Show cluster centers (centroids) indicating mean values of each feature for each cluster

km.cluster_centers_

array([[ -2.80000000e+00,  -1.40000000e+00,   0.00000000e+00,
          9.00000000e-01,  -2.00000000e-01,   3.80000000e+00,
         -1.30000000e+00,  -1.70000000e+00,   8.30000000e+00,
         -2.00000000e-01,  -4.00000000e-01,  -2.30000000e+00,
         -3.50000000e+00],
       [  2.90000000e+00,   1.90000000e+00,   0.00000000e+00,
         -4.00000000e-01,   1.00000000e-01,  -4.00000000e+00,
          1.30000000e+00,   1.60000000e+00,  -6.30000000e+00,
         -3.00000000e-01,   2.84372526e-12,   1.40000000e+00,
          2.30000000e+00],
       [ -9.00000000e-01,  -1.60000000e+00,   0.00000000e+00,
         -2.80000000e+00,  -2.00000000e-01,   1.04000000e+01,
         -3.30000000e+00,  -3.60000000e+00,   1.14000000e+01,
         -1.50000000e+00,  -1.00000000e+00,  -3.10000000e+00,
         -4.60000000e+00],
       [  1.00000000e-01,  -4.00000000e-01,   0.00000000e+00,
          7.00000000e-01,  -2.00000000e-01,  -2.00000000e-01,
         -6.00000000e-01,  -2.20000000e+00,   7.000

In [118]:
# Show cluster labels

my_label = km.labels_

my_label

array([1, 2, 2, ..., 2, 4, 2])

In [119]:
# Show distribution of cluster labels

np.unique(my_label, return_counts=True)



(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 171498,  261200, 1044761,  255376,  273896,  124249,   10328,
          18708], dtype=int64))

In [120]:
# Show distribution of Supergroup labels

np.unique(y, return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 
       dtype='|S1'),
 array([  18708,  273896,  124249, 1044761,  255376,  171498,  261200,
          10328], dtype=int64))

### Features related to Education Levels

In [122]:
features = [50, 51, 52]

num_features = len(features)

#-gather fields of interest from full data set into X and y
X = []

for rec in rawdata:
    instance = []
    for f in features:
        instance.append( float( rec[f] ))
    X.append( instance )
        
X = np.array( X )
num_instances = X.shape[0]

print 'number of instances = %d' % num_instances
print 'shape of input data = %d x %d' % ( X.shape[0], X.shape[1] )



number of instances = 2160016
shape of input data = 2160016 x 3


In [58]:
# Create a dataframe with the results of the Inertia and Calinski-Harabasz scores


columns = ['No. of clusters', 'Inertia', 'Calinski-Harabaz Score']

index = np.arange(14)

df7 = pd.DataFrame(columns = columns, index = index)

line_count = 0

n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 100]

cluster_count = 0

while cluster_count < 14:
    km = cluster.KMeans( n_clusters[cluster_count] ).fit( X )
    labels = km.labels_
    ch_score = metrics.calinski_harabaz_score(X, labels)
    df7['No. of clusters'][line_count] = n_clusters[cluster_count]
    df7['Inertia'][line_count] = km.inertia_
    df7['Calinski-Harabaz Score'][line_count] = ch_score
    cluster_count = cluster_count + 1
    line_count = line_count + 1



In [59]:
# Show the results of the Inertia and Calinski-Harabasz scores

df7

,No. of clusters,Inertia,Calinski-Harabaz Score
0,2,6.45197e+07,1.67556e+07
1,3,8.06272e+06,7.46036e+07
2,4,3.56687e+06,1.13332e+08
3,5,1.59709e+06,1.905e+08
4,6,581551,4.19283e+08
5,7,54816.5,3.71029e+09
6,8,1.12426e-13,9.44617e+26
7,9,1.22269e-15,9.44617e+26
8,10,1.22095e-15,9.44617e+26
9,20,1.1909e-15,9.44617e+26


In [123]:
# Since the above results show that the ideal clustering is 8 clusters, fit the model such that it partitions the data into
# 8 clusters.

km = cluster.KMeans(n_clusters=8).fit(X)

In [124]:
# Show cluster centers (centroids) indicating mean values of each feature for each cluster

km.cluster_centers_

array([[ -8.00000000e-01,  -1.29000000e+01,   2.46000000e+01],
       [  2.15072404e-12,   1.00000000e+00,  -9.40000000e+00],
       [ -3.00000000e-01,  -5.60000000e+00,   1.03000000e+01],
       [ -4.00000000e-01,  -6.10000000e+00,   1.69000000e+01],
       [  2.00000000e-01,   9.30000000e+00,  -1.56000000e+01],
       [ -5.69988501e-13,   1.60000000e+00,  -6.50000000e+00],
       [  1.40000000e+00,   8.80000000e+00,  -5.30000000e+00],
       [ -2.00000000e-01,   1.60000000e+00,  -9.30000000e+00]])

In [125]:
# Show cluster labels

my_label = km.labels_

my_label

array([7, 0, 0, ..., 0, 1, 0])

In [126]:
# Show distribution of cluster labels

np.unique(my_label, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([1044761,  273896,  255376,  171498,   18708,  124249,   10328,
         261200], dtype=int64))

In [127]:
# Show distribution of Supergroup labels

np.unique(y, return_counts=True)

(array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], 
       dtype='|S1'),
 array([  18708,  273896,  124249, 1044761,  255376,  171498,  261200,
          10328], dtype=int64))

## Step 6: Obtain information on the travel patterns of each cluster, including the most frequently used stations, the distribution of travel throughout a day, and the distribution of travel throughout a week

### Merge the Passenger dataframe with the TfL dataframe

In [29]:
# Create a new dataframe from the loac_input_df dataframe that has all the TfL tube and rail stations but discard 
# unnecessary fields like 'Keep', 'Supergroup', and 'id'

station_match_df = loac_input_df.drop('Keep', axis=1)

station_match_df = station_match_df.drop('Supergroup', axis=1)

station_match_df = station_match_df.drop('id', axis=1)


In [30]:
# Show the head of the station_match_df dataframe

station_match_df.head()

,STATIONKEY,STATIONNAME
0,1,Acton Town
1,2,Barbican
2,3,Aldgate
3,4,Aldgate East
4,5,Alperton


In [31]:
# Add a new row that refers to unknown stations

# Source: https://stackoverflow.com/questions/24284342/insert-a-row-to-pandas-dataframe

station_match_df.loc[-1] = [-1, 'Unknown'] # add a new row for unknown station keys (marked as -1)
station_match_df.index = station_match_df.index + 1 # shift the index
station_match_df = station_match_df.sort() # sort the dataframe by index

C:\Users\HP-PC\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [33]:
# Merge the finalmerged_df dataframe (which contains all the passenger journey records from the sample dataset) with the 
# Station Names dataframe so that each Station of Entry Key is matched up with its respective station name.

finalstation_df = finalmerged_df.merge(station_match_df, left_on='STATIONOFFIRSTENTRYKEY', right_on='STATIONKEY', how='left')



In [34]:
finalstation_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock,Day of the Week,STATIONKEY,STATIONNAME
0,11170717,-1,12393,15,247,-1,896,8,Friday,247,Woodford
1,31776450,5,12393,15,213,111,1483,99,Friday,213,Tower Hill
2,6666492,-1,12393,15,76,-1,1126,10,Friday,76,Fulham Broadway
3,58927301,-1,12393,15,158,-1,1333,12,Friday,158,Perivale
4,23724328,-1,12393,15,193,-1,1021,9,Friday,193,South Kensington


In [35]:
# Rename the 'STATIONNAME' column as 'Station of Entry'

finalstation_df = finalstation_df.rename(columns={'STATIONNAME': 'Station of Entry'})

In [36]:
finalstation_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock,Day of the Week,STATIONKEY,Station of Entry
0,11170717,-1,12393,15,247,-1,896,8,Friday,247,Woodford
1,31776450,5,12393,15,213,111,1483,99,Friday,213,Tower Hill
2,6666492,-1,12393,15,76,-1,1126,10,Friday,76,Fulham Broadway
3,58927301,-1,12393,15,158,-1,1333,12,Friday,158,Perivale
4,23724328,-1,12393,15,193,-1,1021,9,Friday,193,South Kensington


In [37]:
# Drop the 'STATIONKEY' column

finalstation_df = finalstation_df.drop('STATIONKEY', axis=1)


In [38]:
finalstation_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock,Day of the Week,Station of Entry
0,11170717,-1,12393,15,247,-1,896,8,Friday,Woodford
1,31776450,5,12393,15,213,111,1483,99,Friday,Tower Hill
2,6666492,-1,12393,15,76,-1,1126,10,Friday,Fulham Broadway
3,58927301,-1,12393,15,158,-1,1333,12,Friday,Perivale
4,23724328,-1,12393,15,193,-1,1021,9,Friday,South Kensington


In [39]:
# Merge the revised finalmerged_df dataframe with the Station Names dataframe so that each Station of Exit Key is matched up with its respective station name.

finalstation_df = finalstation_df.merge(station_match_df, left_on='STATIONOFEXITKEY', right_on='STATIONKEY', how='left')



In [40]:
finalstation_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock,Day of the Week,Station of Entry,STATIONKEY,STATIONNAME
0,11170717,-1,12393,15,247,-1,896,8,Friday,Woodford,-1,Unknown
1,31776450,5,12393,15,213,111,1483,99,Friday,Tower Hill,111,Kilburn
2,6666492,-1,12393,15,76,-1,1126,10,Friday,Fulham Broadway,-1,Unknown
3,58927301,-1,12393,15,158,-1,1333,12,Friday,Perivale,-1,Unknown
4,23724328,-1,12393,15,193,-1,1021,9,Friday,South Kensington,-1,Unknown


In [41]:
# Rename the 'STATIONNAME' column as 'Station of Exit'

finalstation_df = finalstation_df.rename(columns={'STATIONNAME': 'Station of Exit'})

In [42]:
# Drop the 'STATIONKEY' column

finalstation_df = finalstation_df.drop('STATIONKEY', axis=1)


In [43]:
finalstation_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock,Day of the Week,Station of Entry,Station of Exit
0,11170717,-1,12393,15,247,-1,896,8,Friday,Woodford,Unknown
1,31776450,5,12393,15,213,111,1483,99,Friday,Tower Hill,Kilburn
2,6666492,-1,12393,15,76,-1,1126,10,Friday,Fulham Broadway,Unknown
3,58927301,-1,12393,15,158,-1,1333,12,Friday,Perivale,Unknown
4,23724328,-1,12393,15,193,-1,1021,9,Friday,South Kensington,Unknown


In [45]:
passengerfinal_df.head()

,PRESTIGEID,STATIONOFFIRSTENTRYKEY,id,STATIONKEY,STATIONNAME,Keep,Supergroup
0,125,161,154,161,Plaistow,True,G
1,190,70,68,70,Euston Square,True,D
2,247,1980,366,1980,Richmond,True,D
3,355,1955,349,1955,Gunnersbury,True,C
4,376,1968,360,1968,Barking,True,B


In [46]:
# Delete all unnecessary columns from passengerfinal_df to create a simple dataframe with just the prestigeid and its associated Supergroup code

passenger_simple_df = passengerfinal_df.drop('STATIONOFFIRSTENTRYKEY', axis=1)
passenger_simple_df = passenger_simple_df.drop('id', axis=1)
passenger_simple_df = passenger_simple_df.drop('STATIONKEY', axis=1)
passenger_simple_df = passenger_simple_df.drop('STATIONNAME', axis=1)
passenger_simple_df = passenger_simple_df.drop('Keep', axis=1)


In [47]:
passenger_simple_df.head()

,PRESTIGEID,Supergroup
0,125,G
1,190,D
2,247,D
3,355,C
4,376,B


In [48]:
# Merge the Passenger dataframe with the TfL journey dataframe so that each journey record in the original dataset
# has an associated Supergroup code.

tflfinal_df = finalstation_df.merge(passenger_simple_df, left_on='PRESTIGEID', right_on='PRESTIGEID', how='left')



In [49]:
tflfinal_df.head()

,PRESTIGEID,PPTPASSENGERAGEKEY,DAYKEY,CARDTYPEKEY,STATIONOFFIRSTENTRYKEY,STATIONOFEXITKEY,TRANSACTIONTIME,Timeblock,Day of the Week,Station of Entry,Station of Exit,Supergroup
0,11170717,-1,12393,15,247,-1,896,8,Friday,Woodford,Unknown,C
1,31776450,5,12393,15,213,111,1483,99,Friday,Tower Hill,Kilburn,B
2,6666492,-1,12393,15,76,-1,1126,10,Friday,Fulham Broadway,Unknown,C
3,58927301,-1,12393,15,158,-1,1333,12,Friday,Perivale,Unknown,C
4,23724328,-1,12393,15,193,-1,1021,9,Friday,South Kensington,Unknown,D


### Get travel patterns of each Supergroup:

In [ ]:
# The purpose of this section is to determine the travel behaviour (including most frequently used stations, most frequent
# travel times throughout the day, and most frequent days travelled throughout the week) of each Supergroup


In [298]:
# Get the frequency distribution of journeys travelled on each day of the week categorized by Supergroup

supergroup_day = tflfinal_df.groupby(['Supergroup', 'Day of the Week']).size()

In [301]:
# Save the above in an output file

supergroup_day.to_csv("Supergroup_day.csv", sep = '\t')

In [225]:
# Show the frequency distribution of journeys travelled on each day of the week categorized by Supergroup

tflfinal_df.groupby(['Supergroup', 'Day of the Week']).size()

Supergroup  Day of the Week
A           Friday               6751
            Monday               5769
            Saturday             2726
            Sunday               1035
            Thursday             6592
            Tuesday              6592
            Wednesday            6551
B           Friday              97951
            Monday              83109
            Saturday            43853
            Sunday              19055
            Thursday            97033
            Tuesday             94594
            Wednesday           95246
C           Friday              46052
            Monday              39087
            Saturday            18116
            Sunday               7253
            Thursday            45026
            Tuesday             44153
            Wednesday           44590
D           Friday             347786
            Monday             286337
            Saturday           158042
            Sunday              68420
            Thursday  

In [302]:
# Get the frequency distribution of journeys travelled during each time interval (timeblock) in a 24-hour day categorized by 
# Supergroup

supergroup_time = tflfinal_df.groupby(['Supergroup', 'Timeblock']).size()

In [303]:
# Save the above in an output file

supergroup_time.to_csv("Supergroup_time.csv", sep = '\t')

In [227]:
# Show the frequency distribution of journeys travelled during each time interval (timeblock) in a 24-hour day categorized by 
# Supergroup

tflfinal_df.groupby(['Supergroup', 'Timeblock']).size()

Supergroup  Timeblock
A           2                1
            3              871
            4             7295
            5             7720
            6             3601
            7             3099
            8             3295
            9             4531
            10            3243
            11            1387
            12             845
            99             128
B           1                7
            2                8
            3             2723
            4            39370
            5            83976
            6            42693
            7            47393
            8            55650
            9            94741
            10           87828
            11           42902
            12           28375
            99            5175
C           1                3
            2                4
            3             2929
            4            41259
            5            56788
                         ...  
G           8    

In [289]:
# Get the frequency distribution of journeys commencing at each station categorized by Supergroup

# Source: https://stackoverflow.com/questions/35364601/group-by-and-find-top-n-value-counts-pandas

s_entry = tflfinal_df['Station of Entry'].groupby(tflfinal_df['Supergroup']).value_counts()

In [291]:
# Show the frequency distribution of journeys commencing at the five most frequently used stations of entry for each Supergroup

# Source: https://stackoverflow.com/questions/35364601/group-by-and-find-top-n-value-counts-pandas

print s_entry.groupby(level=0).nlargest(5)

Supergroup  Supergroup  Station of Entry    
A           A           Dagenham Heathway        10120
                        Becontree                 6404
                        Elm Park                  5530
                        Dagenham East             4636
                        Ruislip Gardens           1894
B           B           Bond Street              68102
                        Warren Street            32060
                        Mile End                 30659
                        Barking                  27160
                        Whitechapel              26161
C           C           Woodford                 11652
                        Gants Hill               11192
                        Newbury Park             10005
                        Gunnersbury               9729
                        Northolt                  8888
D           D           Oxford Circus           127371
                        Stratford                91850
                    

In [292]:
# Get the frequency distribution of journeys ending at each station categorized by Supergroup

# Source: https://stackoverflow.com/questions/35364601/group-by-and-find-top-n-value-counts-pandas

s_exit = tflfinal_df['Station of Exit'].groupby(tflfinal_df['Supergroup']).value_counts()

In [297]:
# Show the frequency distribution of journeys ending at the five most frequently used stations of exit for each Supergroup

# Source: https://stackoverflow.com/questions/35364601/group-by-and-find-top-n-value-counts-pandas

print s_exit.groupby(level=0).nlargest(6)

Supergroup  Supergroup  Station of Exit     
A           A           Unknown                   21737
                        Barking                    1380
                        Dagenham Heathway          1362
                        Becontree                   946
                        Elm Park                    735
                        Dagenham East               646
B           B           Unknown                  331832
                        Bond Street                7473
                        Oxford Circus              7329
                        Southwark                  5204
                        Stratford                  5049
                        Mile End                   4756
C           C           Unknown                  147351
                        Stratford                  2663
                        Oxford Circus              2466
                        Holborn                    1860
                        Woodford                   1726
   